In [1]:
import pandas as pd
import numpy as np

file_name = '../HashtagNetwork/allseason_20k_rawdata.json'

allseason = pd.read_json(file_name)

In [2]:
base_df = allseason[['id','hashtags']]

In [3]:
base_df.index = base_df['id']

In [4]:
base_df.head()

,id,hashtags
id,,
BtOm-dkHn5v,BtOm-dkHn5v,"[sport, travelling, winterfashion, cestovani, ..."
BtQp-IfHkzq,BtQp-IfHkzq,"[instagood, instastyle, fashionblog, fashionbl..."
BtRdH3UBQ6i,BtRdH3UBQ6i,"[london, like4like, corduroy, beanie, ootd, uk..."
BtRJG92g_CG,BtRJG92g_CG,[]
BtOpNQpApMI,BtOpNQpApMI,"[styleover50, chiberia, styleisforlife, thrift..."


In [5]:
base_df = base_df.drop(columns=['id'])

# TagBlender & hashtaggedapp

In [6]:
before_len = len(base_df)

In [7]:
before_len

19981

In [8]:
blenderlist = []

In [9]:
for id,hashtags in base_df.iterrows():
    tags = hashtags['hashtags']
    
    if 'tagblender' in tags:
        blenderlist.append(id)

In [10]:
len(blenderlist)

25

In [11]:
print(blenderlist)

['BtRPygplZMD', 'BtRMZWsH48L', 'BtQTtVvB96P', 'BtPGr5Tli-r', 'BtPBsgeHnXE', 'BtO_6YonlnU', 'BtPWz0hAVEO', 'BtQhdlagz1B', 'BtRLVkPh4hh', 'BtO_-F7HHo3', 'BtQFOJ5h8xs', 'Bs0lrqlnXwS', 'BtIiSvNHHQg', 'BtGI90HgPMv', 'BtBaNaBFWw_', 'BVxeqg3BTdl', 'Bs5l0uBDctC', 'BrmB4GRDs6z', 'BWDpTI_B0tC', 'Bs-9-TPF96u', 'Bs_pJ8agvm0', 'Bnwn5ivhixz', 'BtB_q3VnoqW', 'Bnd2YFEF5zB', 'Bs_kTASBZfp']


In [12]:
hashtaggedapp_list = []

In [13]:
for id,hashtags in base_df.iterrows():
    tags = hashtags['hashtags']
    
    if 'hashtaggedapp' in tags:
        hashtaggedapp_list.append(id)

In [14]:
len(hashtaggedapp_list)

3

In [15]:
blenderlist += hashtaggedapp_list

In [16]:
for id in blenderlist:
    base_df = base_df.drop([id],0)

In [17]:
base_df.head()

,hashtags
id,
BtOm-dkHn5v,"[sport, travelling, winterfashion, cestovani, ..."
BtQp-IfHkzq,"[instagood, instastyle, fashionblog, fashionbl..."
BtRdH3UBQ6i,"[london, like4like, corduroy, beanie, ootd, uk..."
BtRJG92g_CG,[]
BtOpNQpApMI,"[styleover50, chiberia, styleisforlife, thrift..."


In [18]:
fashion_df = base_df[base_df.astype(str)['hashtags'] != '[]']

In [19]:
base_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19953 entries, BtOm-dkHn5v to BtJVaqsDGgY
Data columns (total 1 columns):
hashtags    19953 non-null object
dtypes: object(1)
memory usage: 311.8+ KB


In [20]:
fashion_df.head()

,hashtags
id,
BtOm-dkHn5v,"[sport, travelling, winterfashion, cestovani, ..."
BtQp-IfHkzq,"[instagood, instastyle, fashionblog, fashionbl..."
BtRdH3UBQ6i,"[london, like4like, corduroy, beanie, ootd, uk..."
BtOpNQpApMI,"[styleover50, chiberia, styleisforlife, thrift..."
BtRHyjZAem6,"[lasvegas, style, mariadelosangeles_mdla, inst..."


In [21]:
fashion_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17616 entries, BtOm-dkHn5v to BtCTPK7AJlC
Data columns (total 1 columns):
hashtags    17616 non-null object
dtypes: object(1)
memory usage: 275.2+ KB


# Testing...

In [22]:
def get_DataFrame_for_affinity(df,taglist):
    
    result_df = pd.DataFrame(
        0, index = np.arange(0,len(df)),columns=taglist)    
    
    for i in range(len(df)):
        list_of_row = list(df.iloc[i])[0]
        
        for tag in list_of_row:
            if tag in taglist:
                result_df[tag][i] = 1
        
    return result_df

In [23]:
def getTagsDict(df):
    tag_dict = {}
    
    for index, row in df.iterrows():
        for tag in row['hashtags']:
            if tag in tag_dict:
                tag_dict[tag] = tag_dict[tag] + 1
            else:
                tag_dict[tag] = 1
                
    sort_dict = sorted(tag_dict.items(), key=lambda kv: kv[1])
    return sort_dict

In [24]:
fashion_dict = getTagsDict(fashion_df)

In [25]:
fashion_dict = fashion_dict[-1000:]

In [27]:
tag_list1000 = []

for tag in fashion_dict:
    tag_list1000.append(tag[0])

In [29]:
before_corr = get_DataFrame_for_affinity(fashion_df,tag_list1000)

In [30]:
corr_affinity = before_corr.corr(method='pearson')

In [32]:
def addone_dividetwo(x):
    return pow((x+1)/2, 2)

In [33]:
corr_affinity_threshold = corr_affinity.copy()
corr_affinity_threshold = corr_affinity_threshold.applymap(addone_dividetwo)

In [35]:
from sklearn.cluster import SpectralClustering

sc = SpectralClustering(n_clusters=50, affinity='precomputed')

In [36]:
predict = pd.DataFrame(sc.fit_predict(corr_affinity_threshold))

In [37]:
predict.columns = ['Group']

In [40]:
node_df = pd.DataFrame(fashion_dict,columns=['Tag','Freq'])

In [42]:
node_df['Group'] = predict

In [43]:
node_df

,Tag,Freq,Group
0,curvywomen,43,11
1,newin,43,37
2,izmir,43,18
3,beachvibes,43,9
4,1950sfashion,43,21
5,australiandesigner,43,37
6,home,44,37
7,summerwear,44,37
8,hijabstyle,44,47
9,natural,44,37


In [45]:
group_sort = node_df.sort_values(['Group'],ascending=True)

,Tag,Freq,Group
543,londonfashion,94,0
441,dubaifashion,77,0
453,tokyo,78,0
775,london,182,0
768,paris,176,0
861,nyc,279,0
701,miami,138,0
744,fashioninsta,161,0
554,glamour,95,0
866,newyork,298,0


In [47]:
groupby_tags2 = group_sort.groupby('Group')['Tag'].apply(list)

In [49]:
summerlist = []
for list in groupby_tags2:
    print(list)
    print()
    if list[0] == 'summeroutfit':
        summerlist = list

['londonfashion', 'dubaifashion', 'tokyo', 'london', 'paris', 'nyc', 'miami', 'fashioninsta', 'glamour', 'newyork']

['happiness', 'hoodies', 'motivation', 'lifestyle', 'creative', 'design', 'polishgirl', 'gym', 'hats', 'igers', 'sunday', 'clothingbrand', 'fitness', 'hoodie', 'vscocam', 'instamood', 'bestoftheday', 'mood', 'friday', 'nofilter', 'clothingline', 'workout', 'gay', 'fashionkilla', 'fitnessmotivation']

['outfitgoals', 'styleinspiration', 'fashionstatement', 'theeverygirl', 'lotd', 'stylediaries', 'outfits', 'wiw', 'dailylook', 'styleblog', 'everydaystyle', 'mystyle', 'ootdmagazine', 'mylook', 'personalstyle', 'stylepost', 'outfitpost', 'igstyle', 'realoutfitgram', 'ootdshare', 'aboutalook', 'whatiworetoday', 'whattowear', 'stylediary', 'whatimwearing', 'pursuepretty', 'lookoftheday', 'flashesofdelight', 'ootdgals', 'ootdfashion', 'stylegoals', 'outfitoftheday', 'outfitstyle', 'indianfashion', 'ootdfash', 'liketoknowit', 'styletips', 'ootd', 'whowhatwear', 'outfitinspo', 'm

In [51]:
summerlist

['summeroutfit',
 'summer',
 'sea',
 'swimwear',
 'bikini',
 'swimsuit',
 'summerbody',
 'summerdays',
 'ocean',
 'beachbody',
 'summerlove',
 'beach',
 'summervibes',
 'beachfashion',
 'summerfashion',
 'bikinigirl',
 'beachwear',
 'beachvibes',
 'beachlife',
 'summertime',
 'summerstyle']

In [52]:
post_id = fashion_df.index.values

In [53]:
summer_onehot = pd.DataFrame(before_corr.loc[0])
summer_onehot = summer_onehot.T
summer_onehot

,curvywomen,newin,izmir,beachvibes,1950sfashion,australiandesigner,home,summerwear,hijabstyle,natural,...,fashionista,instafashion,fashionblogger,ootd,style,fallfashion,summerfashion,springfashion,winterfashion,fashion
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [54]:
for tag in tag_list1000:
    summer_onehot[tag] = 0

for tag in summerlist:
    summer_onehot[tag] = 1
    
summer_onehot

,curvywomen,newin,izmir,beachvibes,1950sfashion,australiandesigner,home,summerwear,hijabstyle,natural,...,fashionista,instafashion,fashionblogger,ootd,style,fallfashion,summerfashion,springfashion,winterfashion,fashion
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [55]:
resultlist = []
for index, row in before_corr.iterrows():
    logical_and = row & summer_onehot
    resultlist.append(logical_and)

In [56]:
sumlist = []
for row in resultlist:
    sumvalue = row.sum(axis = 1)
    sumlist.append(sumvalue)

In [57]:
resultpd = pd.DataFrame(sumlist, post_id)

In [58]:
resultpd.sort_values(by=0, ascending = False)

,0
BtMYtdTneYP,11
BtN9MlGHV_k,11
BtAaNicgtt4,11
Bs-Y0nVhhR6,11
BtN9YVTHXmc,11
BtBV7LvguXP,11
BtIdVcWgyk4,11
BtOpe9infvQ,11
BtN9TfSH4Hu,11
BtNyaLSHNpG,11
